<a href="https://colab.research.google.com/github/shatlykgurdov/3.1.2/blob/main/5b-c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# GWP1 — Problem 5b and 5c (Notebook Code)
# NOTE: Put this code in your Jupyter/Colab notebook.
# DO NOT paste this code into the PDF report.
# ============================================================

# -------------------------
# Common imports (used in 5b and 5c)
# -------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller

# ============================================================
# 5b) Use real economic data for equities and illustrate if the
#     time series you chose has a unit root.
# ============================================================

# ---- 5b.1: Load real equity price data ----
# Option A (recommended): use yfinance (works in Colab / many environments)
# If yfinance is not installed, uncomment the next line:
# !pip -q install yfinance

import yfinance as yf

TICKER = "AAPL"            # change if you want: "MSFT", "^GSPC" (S&P500), "TSLA", etc.
START = "2018-01-01"
END   = "2024-01-01"

data = yf.download(TICKER, start=START, end=END, progress=False)
prices = data["Close"].dropna()

print(f"5b) Loaded {len(prices)} daily closing prices for {TICKER} from {prices.index.min().date()} to {prices.index.max().date()}")

# ---- 5b.2: Plot the price series ----
plt.figure(figsize=(10, 4))
plt.plot(prices)
plt.title(f"5b) {TICKER} Daily Closing Prices")
plt.xlabel("Date")
plt.ylabel("Price")
plt.tight_layout()
plt.show()

# ---- 5b.3: Run ADF test on levels (prices) ----
# H0: unit root (non-stationary)
# H1: stationary
adf_levels = adfuller(prices, autolag="AIC")

print("\n5b) ADF test on PRICE LEVELS")
print("ADF statistic:", adf_levels[0])
print("p-value      :", adf_levels[1])
print("Used lags     :", adf_levels[2])
print("N obs         :", adf_levels[3])
print("Critical vals :", adf_levels[4])

if adf_levels[1] < 0.05:
    print("Conclusion (5%): Reject H0 → evidence against unit root (stationary).")
else:
    print("Conclusion (5%): Fail to reject H0 → consistent with a unit root (non-stationary).")

# ---- 5b.4 (optional but very helpful): show that first differences are usually stationary ----
returns_diff = prices.diff().dropna()  # first difference of price (not log returns)
adf_diff = adfuller(returns_diff, autolag="AIC")

plt.figure(figsize=(10, 4))
plt.plot(returns_diff)
plt.title(f"5b) First Difference of {TICKER} Prices")
plt.xlabel("Date")
plt.ylabel("Δ Price")
plt.tight_layout()
plt.show()

print("\n5b) ADF test on FIRST DIFFERENCES (ΔPrice)")
print("ADF statistic:", adf_diff[0])
print("p-value      :", adf_diff[1])
print("Critical vals :", adf_diff[4])

if adf_diff[1] < 0.05:
    print("Conclusion (5%): Reject H0 → differenced series is likely stationary.")
else:
    print("Conclusion (5%): Fail to reject H0 → differenced series may still be non-stationary.")

# ============================================================
# 5c) Why are we concerned about a unit root (ρ=1) and not a
#     root of 1.5? Use simulations to compare the two.
# ============================================================

np.random.seed(42)

# ---- 5c.1: Simulation settings ----
T = 200                # number of time points
sigma_eps = 1.0        # shock volatility

# Unit root (random walk): y_t = y_{t-1} + eps_t   (ρ = 1)
rho_unit = 1.0

# Explosive root: y_t = ρ y_{t-1} + eps_t          (ρ = 1.5)
rho_explosive = 1.5

eps = np.random.normal(0, sigma_eps, size=T)

# ---- 5c.2: Generate the two processes ----
y_unit = np.zeros(T)
y_explosive = np.zeros(T)

for t in range(1, T):
    y_unit[t] = rho_unit * y_unit[t-1] + eps[t]         # random walk
    y_explosive[t] = rho_explosive * y_explosive[t-1] + eps[t]  # explosive

# ---- 5c.3: Plot both series for visual comparison ----
plt.figure(figsize=(10, 4))
plt.plot(y_unit, label="5c) Unit Root (ρ = 1)")
plt.plot(y_explosive, label="5c) Explosive Root (ρ = 1.5)")
plt.title("5c) Simulated Time Series: Unit Root vs Explosive Root")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.tight_layout()
plt.show()

# ---- 5c.4: Optional extra plot (often clearer): same plot but with y-axis limits for unit root ----
# This helps show how explosive series quickly dominates the scale.
plt.figure(figsize=(10, 4))
plt.plot(y_unit, label="Unit Root (ρ = 1)")
plt.title("5c) Unit Root Process Alone (ρ = 1)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.tight_layout()
plt.show()

print("\n5c) Notes for interpretation:")
print("- ρ = 1 (unit root) behaves like a random walk: shocks have permanent effects, series wanders but does not explode deterministically.")
print("- ρ = 1.5 is explosive: values grow rapidly in magnitude, typically not resembling real financial/economic series over long samples.")
print("- This is why unit roots are a key concern: they create non-stationarity without unrealistic explosive growth.")
